<a href="https://colab.research.google.com/github/gouravgujariya/machine-learning-project/blob/main/fashion_minst_deeplearning_hyperparametertunner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras

In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 7.8 MB/s 


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_img,train_labels),(test_img,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


scalling down the dataset [0:1] to quickly train model

In [6]:
train_img=train_img/255.0
test_img=test_img/255.0

this is how dataset looks like

In [7]:
train_img[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [8]:
train_img[0].shape

(28, 28)

In [9]:
train_img=train_img.reshape(len(train_img),28,28,1)
test_img=test_img.reshape(len(test_img),28,28,1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [12]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [14]:
tuner_search.search(train_img,train_labels,epochs=3,validation_split=0.1)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
96                |?                 |conv_1_filter
5                 |?                 |conv_1_kernel
64                |?                 |conv_2_filter
5                 |?                 |conv_2_kernel
96                |?                 |dense_1_units
0.001             |?                 |learning_rate

Epoch 1/3
1688/1688 [==============================] - 567s 336ms/step - loss: 0.4095 - accuracy: 0.8509 - val_loss: 0.3328 - val_accuracy: 0.8817
Epoch 2/3
 117/1688 [=>............................] - ETA: 8:35 - loss: 0.2829 - accuracy: 0.8974

KeyboardInterrupt: ignored

In [14]:
model=tuner_search.get_best_models(num_models=1)[0]

In [15]:
model.fit(train_img, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1279 - accuracy: 0.9529 - val_loss: 0.2805 - val_accuracy: 0.9138
Epoch 5/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0924 - accuracy: 0.9650 - val_loss: 0.3017 - val_accuracy: 0.9165
Epoch 6/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0658 - accuracy: 0.9755 - val_loss: 0.3623 - val_accuracy: 0.9158
Epoch 7/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0499 - accuracy: 0.9818 - val_loss: 0.3748 - val_accuracy: 0.9113
Epoch 8/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0349 - accuracy: 0.9873 - val_loss: 0.4893 - val_accuracy: 0.9108
Epoch 9/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0292 - accuracy: 0.9893 - val_loss: 0.5048 - val_accuracy: 0.9108
Epoch 10/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0271 - accuracy: 0.9904 - val_loss: 0.5183 - val_a